# LAB 2

Networks and Systems Security
Week 02
Foundations of Network Security

### Setting up
Install the cryptography libary

In [2]:
!pip install cryptography

### Step 1: Key Generation (Run Once)

Keys are used in security to encrypt, decrypt, sign and verify data using two main methods.
- Symmetric keys: Where the same key is used and capable of encrypting and decrypting data.
- Asymmetric keys: Where mathmatically linked private and public keys are used to encrypt and decrypt data, much more secure, garanteeing confidentiality, authenticity and integrity.

code

In [1]:
# generate_keys.py
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# Generate private key
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)

# Save private key
with open("private_key.pem", "wb") as f:
    f.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )
    )
    
# Save public key
public_key = private_key.public_key()
with open("public_key.pem", "wb") as f:
    f.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )   
    )
print("✅ Keys saved: private_key.pem, public_key.pem")

✅ Keys saved: private_key.pem, public_key.pem


### Analysis
We generate a public and private RSA key pair, saving them onto seporate files, private_key.pem and public_key.pem. These are a long blob of mathmatically linked base64 encoded data.

### Interpretation 
This demonstrates the foundation of asymmetric cryptography, where anything encrypted with the public key can only be decrypted by the private key, and anything signed with the private key can be verified with the public key. This being a fundamental concept in security. This is how many real systems work, such as email encryption or HTTPS certificates.

### Step 2: Receiver (Server) – Receive & Decrypt

A server is usually used to manage data via waiting, accepting, recieving, processing and responding to data.

There are different ways programs communicate over a network, including using UDP and TCP. TCP is slower, but more reliable communication method compared to UDP, ensuring all data arrives in order, and that there is no lost, duplicate or currupted packets, making it suitable for secure message transfer.

code

In [6]:
# receiver.py
import socket
import pickle
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# Start server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("localhost", 65432))
    s.listen()
    print(" Waiting for connection...")
    conn, addr = s.accept()
    with conn:
        print(f" Connected by {addr}")
        data = b""
        while True:
            chunk = conn.recv(4096)
            if not chunk:
                break
            data += chunk

# Unpack payload
encrypted_key, iv, encrypted_message = pickle.loads(data)

# 1. Decrypt AES key with RSA private key
aes_key = private_key.decrypt(
    encrypted_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 2. Decrypt message with AES
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
decryptor = cipher.decryptor()
message = decryptor.update(encrypted_message) + decryptor.finalize()

print(" Decrypted message:", message.decode())

 Waiting for connection...
 Connected by ('127.0.0.1', 1366)
 Decrypted message: Hello from the secure sender! This is confidential.


### Analysis
Here the server loads its private key (for decreption), then opens a TCP socket, waiting for a client to connect. Once a connection is established it receives the encrypted data, and then decrypts it using its private key, displaying the connected connection and decrypted message as a simple string.

### Interpretation

This again demonstrates asymmetric encryption, where only the server—holding the private key—can recover the symmetric key that the client encrypted with the public key.

## Connecting to another device

code

In [5]:
# receiver.py
import socket
import pickle
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(f.read(), password=None)

# Start server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(("0.0.0.0", 65432)) # listen on all interfaces
    s.listen()
    print(" Waiting for connection...")
    conn, addr = s.accept()
    with conn:
        print(f" Connected by {addr}")
        data = b""
        while True:
            chunk = conn.recv(4096)
            if not chunk:
                break
            data += chunk

# Unpack payload
encrypted_key, iv, encrypted_message = pickle.loads(data)

# 1. Decrypt AES key with RSA private key
aes_key = private_key.decrypt(
    encrypted_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# 2. Decrypt message with AES
cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv))
decryptor = cipher.decryptor()
message = decryptor.update(encrypted_message) + decryptor.finalize()

print(" Decrypted message:", message.decode())

 Waiting for connection...
 Connected by ('172.25.128.165', 51168)
 Decrypted message: Hello Jeremi, did you get my message?


#### Analysis

To send a message to my friend on another device, we had to change "localhost" for both reciever and sender to actual ips used across a network. For the server we use 0.0.0.0 which simply means it will accept incoming connections from any interface, whilst for the sender we use the local lan ip. We then had to share our public keys to be able to decrypt the messages sent between each other. (You should never share your private key!)

#### Mini reflection

We originally tried to do this over the school network, but kept recieving errors and time outs. This is likely thanks to this network having extra security protection, stopping our messages from being sent over it. We solved this issue by using a hotspot we could both connect to, using its local ip instead.

#### Outcome

With that we were successful at sending messages over this network between each other. This being same model used for secure messaging apps like WhatsApp, encrypted emails, and VPN tunnels.

